<a href="https://colab.research.google.com/github/sediri-code/spam_detection/blob/main/Spamdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#numpy
import numpy as np
#change the current directory
import os
os.chdir("/content/sample_data/")
#importation of the corpus
import pandas as pd

# Use pd instead of pandas for the alias
spams = pd.read_table("SMSSpamCollection.txt", sep="\t", header=0)



In [9]:
#type of the object
print(type(spams))
#size of the dataset
print(spams.shape)

<class 'pandas.core.frame.DataFrame'>
(5572, 2)


In [12]:
#list of columns
print(spams.columns)
#type of columns
print(spams.dtypes)
#description
print(spams.describe())



Index(['classe', 'message'], dtype='object')
classe     object
message    object
dtype: object
       classe                 message
count    5572                    5572
unique      2                    5169
top       ham  Sorry, I'll call later
freq     4825                      30


In [14]:
crosstab_result = pd.crosstab(index=spams["classe"], columns="count")
print(crosstab_result)

col_0   count
classe       
ham      4825
spam      747


In [16]:
from sklearn.model_selection import train_test_split
import pandas as pd  # Import pandas with the alias pd

# Splitting the dataset into train and test sets
train_size = 3572  # The desired number of samples in the training set
test_size = len(spams) - train_size  # The remaining samples go to the test set

spamsTrain, spamsTest = train_test_split(spams, test_size=test_size, random_state=1, stratify=spams['classe'])

# Frequency distribution of the class attribute for the training set
freqTrain = pd.crosstab(index=spamsTrain["classe"], columns="count")
print(freqTrain / freqTrain.sum())

# Frequency distribution of the class attribute for the test set
freqTest = pd.crosstab(index=spamsTest["classe"], columns="count")
print(freqTest / freqTest.sum())


col_0      count
classe          
ham     0.865901
spam    0.134099
col_0   count
classe       
ham     0.866
spam    0.134


In [17]:
#import the CountVectorizer tool
from sklearn.feature_extraction.text import CountVectorizer
#instantiation of the objet – binary weighting
parseur = CountVectorizer(binary=True)
#create the document term matrix
XTrain = parseur.fit_transform(spamsTrain['message'])

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
tokens = parseur.get_feature_names_out()  # To get the list of tokens
num_tokens = len(tokens)  # To get the number of tokens

print("Number of tokens:", num_tokens)
print("List of tokens:", tokens)

Number of tokens: 6789
List of tokens: ['00' '000' '000pes' ... 'zouk' 'zyada' '〨ud']


In [21]:
#transform the sparse matrix into a numpy matrix
mdtTrain = XTrain.toarray()
#type of the matrix
print(type(mdtTrain))
#size of the matrix
print(mdtTrain.shape)

<class 'numpy.ndarray'>
(3572, 6789)


In [23]:
freq_mots = np.sum(mdtTrain, axis=0)

# Sort the indices of terms by term frequencies
index = np.argsort(freq_mots)

# Get the terms and their frequencies, and create a DataFrame
term_names = np.asarray(parseur.get_feature_names_out())[index]
term_frequencies = freq_mots[index]
df = pd.DataFrame({'terme': term_names, 'freq': term_frequencies})

# Print the DataFrame
print(df)

         terme  freq
0        kaiez     1
1         mary     1
2     marriage     1
3         maps     1
4       manual     1
...        ...   ...
6784       and   522
6785        in   528
6786       the   647
6787       you  1042
6788        to  1091

[6789 rows x 2 columns]


In [24]:
#import the class LogistiRegression
from sklearn.linear_model import LogisticRegression
#instatiate the object
modelFirst = LogisticRegression()
#perform the training process
modelFirst.fit(mdtTrain,spamsTrain['classe'])

LogisticRegression()

In [26]:
#size of coefficients matrix
print(modelFirst.coef_.shape)
#intercept of the model
print(modelFirst.intercept_)

(1, 6789)
[-4.9643061]


In [27]:
#create the document term matrix
mdtTest = parseur.transform(spamsTest['message'])
#size of the matrix
print(mdtTest.shape)

(2000, 6789)


In [28]:
#prediction for the test set
predTest = modelFirst.predict(mdtTest)

In [29]:
#import the metrics class for the performance measurement
from sklearn import metrics
#confusion matrix
mcTest = metrics.confusion_matrix(spamsTest['classe'],predTest)
print(mcTest)
#recall
print(metrics.recall_score(spamsTest['classe'],predTest,pos_label='spam'))
#precision
print(metrics.precision_score(spamsTest['classe'],predTest,pos_label='spam'))
#F1-Score
print(metrics.f1_score(spamsTest['classe'],predTest,pos_label='spam'))
#accuracy rate
print(metrics.accuracy_score(spamsTest['classe'],predTest))

[[1731    1]
 [  40  228]]
0.8507462686567164
0.9956331877729258
0.9175050301810864
0.9795


In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Create a new CountVectorizer with the specified options
parseurBis = CountVectorizer(stop_words='english', binary=True, min_df=10)

# Fit the new CountVectorizer on the training data and transform it into a document-term matrix
XTrainBis = parseurBis.fit_transform(spamsTrain['message'])

# Get the number of tokens
num_tokens = len(parseurBis.get_feature_names_out())

# Instantiate the LogisticRegression model
modelBis = LogisticRegression()


In [35]:
# Create the document-term matrix for the training set
mdtTrainBis = parseurBis.transform(spamsTrain['message'])

# Perform the training process
modelBis.fit(mdtTrainBis, spamsTrain['classe'])

# Create the document-term matrix for the test set
mdtTestBis = parseurBis.transform(spamsTest['message'])

# Make predictions for the test set
predTestBis = modelBis.predict(mdtTestBis)

# Import necessary metrics library
from sklearn import metrics

# Confusion matrix
mcTestBis = metrics.confusion_matrix(spamsTest['classe'], predTestBis)
print("Confusion Matrix:\n", mcTestBis)

# Recall
recall = metrics.recall_score(spamsTest['classe'], predTestBis, pos_label='spam')
print("Recall (True Positive Rate):", recall)

# Precision
precision = metrics.precision_score(spamsTest['classe'], predTestBis, pos_label='spam')
print("Precision:", precision)

# F1-Score
f1_score = metrics.f1_score(spamsTest['classe'], predTestBis, pos_label='spam')
print("F1-Score:", f1_score)

# Accuracy Rate
accuracy = metrics.accuracy_score(spamsTest['classe'], predTestBis)
print("Accuracy:", accuracy)


Confusion Matrix:
 [[1731    1]
 [  37  231]]
Recall (True Positive Rate): 0.8619402985074627
Precision: 0.9956896551724138
F1-Score: 0.9239999999999999
Accuracy: 0.981


In [36]:
#absolute value of the coefficients
coef_abs = np.abs(modelBis.coef_[0,:])
#percentiles of the coefficients (absolute value)
thresholds = np.percentile(coef_abs,[0,25,50,75,90,100])
print(thresholds)

[2.37540153e-03 1.77540082e-01 2.95911402e-01 6.09779140e-01
 1.05613226e+00 2.73460362e+00]


In [37]:
#identify the coefficients "significantly" higher than zero
#use 1st quartile as threshold
indices = np.where(coef_abs > thresholds[1])
print(len(indices[0]))

405


In [38]:
#train and test sets
mdtTrainTer = mdtTrainBis[:,indices[0]]
mdtTestTer = mdtTestBis[:,indices[0]]
#checking
print(mdtTrainTer.shape)
print(mdtTestTer.shape)

(3572, 405)
(2000, 405)


In [39]:
#instatiate the object
modelTer = LogisticRegression()
#train a new classifier with selected terms
modelTer.fit(mdtTrainTer,spamsTrain['classe'])
#prediction on the test set
predTestTer = modelTer.predict(mdtTestTer)
#confusion matrix
mcTestTer = metrics.confusion_matrix(spamsTest['classe'],predTestTer)
print(mcTestTer)
#recall
print(metrics.recall_score(spamsTest['classe'],predTestTer,pos_label='spam'))
#precision
print(metrics.precision_score(spamsTest['classe'],predTestTer,pos_label='spam'))
#F1-Score
print(metrics.f1_score(spamsTest['classe'],predTestTer,pos_label='spam'))
#accuracy rate
print(metrics.accuracy_score(spamsTest['classe'],predTestTer))

[[1731    1]
 [  37  231]]
0.8619402985074627
0.9956896551724138
0.9239999999999999
0.981


In [41]:
# Assuming you want to select and sort terms based on the modelBis
# Replace modelTer with modelBis in your code

# Sorted indices of the absolute value coefficients
sorted_indices = np.argsort(np.abs(modelBis.coef_[0, :]))

# Selected terms based on the sorted indices
sel_terms = np.array(parseurBis.get_feature_names_out())[sorted_indices]

# Terms and their coefficients
term_coefficients = modelBis.coef_[0, :][sorted_indices]

# Create a DataFrame with terms and their coefficients
term_coef_df = pd.DataFrame({'term': sel_terms, 'coef': term_coefficients})

print(term_coef_df)


        term      coef
0       year -0.002375
1       stay -0.009559
2       just -0.012587
3      press -0.017661
4         bt -0.020953
..       ...       ...
536      new  2.036576
537     150p  2.059258
538  service  2.202718
539    claim  2.257584
540       uk  2.734604

[541 rows x 2 columns]


In [42]:
#document to classify
doc = ['this is a new free service for you only']
#get its description
desc = parseurBis.transform(doc)
print(desc)

  (0, 166)	1
  (0, 315)	1
  (0, 405)	1


In [45]:
# Assuming you have a list of valid integer indices in 'selected_indices'
selected_indices = [0, 1, 2, 3, 4]  # Replace with your actual integer indices

# Retrieve the terms based on the selected indices
selected_terms = np.array(parseurBis.get_feature_names_out())[selected_indices]

# Print the selected terms
print(selected_terms)



['000' '08000839402' '10' '100' '1000']


In [46]:
#dense representation
dense_desc = desc.toarray()
#apply var. selection
dense_sel = dense_desc[:,indices[0]]

In [47]:
#prediction of the class membership
pred_doc = modelTer.predict(dense_sel)
print(pred_doc)

['spam']


In [48]:
#prediction of the class membership probabilities
pred_proba = modelTer.predict_proba(dense_sel)
print(pred_proba)

[[0.07816085 0.92183915]]


In [49]:
#checking - logit
logit = 1.823208 + 1.999089 + 2.201104 + modelTer.intercept_
#probability – logistic function
import math
p_spam = 1/(1+math.exp(-logit))
print(p_spam)

0.9141017335167014
